In [2]:
import pandas as pd

In [3]:
pd.__version__

'1.4.2'

In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df.shape[1]

19

In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [7]:
td = df.duration.iloc[0]

In [8]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [11]:
df['duration'].std(ddof=1)

42.594351241920904

In [34]:
dfc = df[(df.duration >= 1) & (df.duration <= 60)]

In [36]:
fraction_left = (len (dfc) / len(df))

In [37]:
print(f"{fraction_left:.2%} of records remain after trimming")

98.12% of records remain after trimming


In [40]:
mask = df['duration'].between(1, 60, inclusive='both')
mask.mean() * 100

98.1220282212598